## Import packages

In [1]:
# import torch
# import numpy as np
# import pandas as pd
# from torch.utils.data import Dataset, DataLoader
# from torch import nn
# import matplotlib.pyplot as plt
# import everything in neural_networks.ipynb (considering it is a notebook)
from ipynb.fs.full.neural_networks import *


## Class definitions

### Dataset
Class that will represent our sampled data. As it inherits from Dataset (torch module), we need to define __ len __ (defines number of samples) and __ getitem __ (defines how to "get" a sample) methods. A Dataset object is then passed to a DataLoader one, which handles loading the data (including possible shuffling)

In [2]:
class DemandDataset(Dataset):
    """
    Dataset class for demand data
    """
    def __init__(self, demand_parameters, problem_params, data_params, seed=None):
        """
        demand_parameters: dict, parameters for demand data
        problem_params: dict, parameters for the problem
        data_params: dict, parameters for the data
        seed: int, seed for random number generator
        """

        self.demand_parameters = demand_parameters
        self.problem_params = problem_params
        self.data_params = data_params
        self.periods = self.problem_params['periods']
        self.num_samples = self.data_params['num_samples']
        self.seed = seed

        demand_generator_functions = {"normal": self.generate_normal_demand_for_one_store}

        self.data = demand_generator_functions[demand_parameters['distribution']](demand_parameters)
    
    
    def generate_normal_demand_for_one_store(self, demand_parameters, **kwargs):
        """
        Generate normal demand data for one store
        """

        # set seed
        if self.seed is not None:
            np.random.seed(self.seed)
        # generate normal distribution, and truncate at 0 from below
        data = np.random.normal(demand_parameters['mean'], 
                                demand_parameters['std'], 
                                size=(self.num_samples, 1, self.periods)
                                )
        # print(data.shape)

        return data
        

    def generate_data(self, demand_parameters, **kwargs):
        """
        Generate demand data
        """
        demand_generator_functions = {"normal": self.generate_normal_demand_for_one_store}
        demand = demand_generator_functions[demand_parameters['distribution']](demand_parameters, **kwargs)
        
        if kwargs['clip']:
            data = np.clip(data, 0, None)

        return 
    
    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        return self.data[idx]

### Fully connected neural network


In [3]:
# class FullyConnectedNN(nn.Module):
#     """
#     Fully connected neural network
#     """

#     def __init__(self, neurons_per_hidden_layer, output_size):
#         """
#         Arguments:
#             neurons_per_hidden_layer: list
#                 list of integers, where each integer is the number of neurons in a hidden layer
#         """

#         super().__init__() # initialize super class

#         # define layers
#         self.activation_function = nn.ELU()
#         self.layers = []
#         for output_neurons in neurons_per_hidden_layer:
#             self.layers.append(nn.LazyLinear(output_neurons))
#             self.layers.append(self.activation_function)
#         self.layers.append(nn.LazyLinear(output_size))
        
#         # define network as a sequence of layers
#         self.net = nn.Sequential(*self.layers)
    
#     def forward(self, state):
#         """
#         Forward pass
#         """
#         x = state['x']
#         # flatten input, except for the batch dimension
#         # x = x.view(x.size(0), -1)
#         x = x.flatten(start_dim=1)
#         # print(f'x.device: {x.device}')
#         # print(f'x.shape: {x.shape}')
#         # add 1.0 to the output of the network to ensure that the output is positive at initialization
#         # otherwise, when applying the clip operator (equivalently, the positive part operator), we won't 'get a gradient'
#         # x = self.net(x)[:, 0] + 1.0
#         x = self.net(x) + 1.0

#         return torch.clip(x, min=0) # clip output to be non-negative

In [4]:
class StateHandler():

    def __init__(self):
        pass

    def transform_state_to_input(self, state):
        """
        Transform state to input for the neural network
        """
        return state

### Simulator class
Object that represents the environment

In [5]:
class Simulator(nn.Module):
    """
    Simulator class
    """

    def __init__(self, parameters, device='cpu'):
        """
        Arguments:
            parameters: dict
                dictionary containing the parameters of the environment
        """

        super().__init__() # initialize super class
        
        self.device = device
        self.parameters = parameters
        self.periods = parameters['periods']
        self.underage_cost = torch.tensor(parameters['underage_cost'])
        self.holding_cost = torch.tensor(parameters['holding_cost'])
        self.lead_time = parameters['lead_time']
        self.lost_demand = parameters['lost_demand']
    
    def move_columns_left(self, tensor_to_displace, start_index, end_index):
        """
        move all columns in given array to the left, and return as list
        """

        return [tensor_to_displace[:, :, i + 1] for i in range(start_index, end_index)]
        # return [tensor_to_displace[:, i + 1] for i in range(start_index, end_index)]

    
    def update_state(self, state, action, demand):
        """
        Update the state of the environment
        """
        inventory_on_hand = state[:, :,  0] - demand
        if self.lost_demand:
            inventory_on_hand = torch.clip(inventory_on_hand, min=0)
        

        # print(f'inventory on hand: {inventory_on_hand.shape}')
        # print(f'state: {state.shape}')
        # print(f'self.move_columns_left(state, 1, self.lead_time - 1).shape: {self.move_columns_left(state, 1, self.lead_time - 1)[0].shape}')
        # print(f'action: {action.shape}')
        # print()
        
        return torch.stack([inventory_on_hand + state[:, :, 1],
                            *self.move_columns_left(state, 1, self.lead_time - 1),
                            action],
                            dim=2
                            )

    # def update_state(self, state, action, demand):
    #     """
    #     Update the state of the environment
    #     """
    #     inventory_on_hand = state[:, 0] - demand
    #     if self.lost_demand:
    #         inventory_on_hand = torch.clip(inventory_on_hand, min=0)
        
    #     return torch.stack([inventory_on_hand + state[:, 1],
    #                         *self.move_columns_left(state, 1, self.lead_time - 1),
    #                         action],
    #                         dim=1
    #                         )
    
    # def update_inventory_for_long_lead_time(self, inventory_on_hand, inventory, lead_time, allocation):

    #     return torch.stack([inventory_on_hand + inventory[:, :, 1],
    #                         *self.move_columns_left(inventory, 1, lead_time - 1),
    #                         allocation],
    #                        dim=2)
    
    def step(self, state, action, demand):
        """
        Simulate one step in the environment
        """
        # get inventory on hand (i.e., at the store before demand arrives)
        inventory_on_hand = state[:, :, 0]
        # print(f'self.underage_cost: {self.underage_cost.device}')
        # print(f'self.holding_cost: {self.holding_cost.device}')
        # print(f'inventory_on_hand: {inventory_on_hand.device}')
        # print(f'demand: {demand.device}')
        # calculate reward as sum of underage and holding costs
        reward = self.underage_cost * torch.clip(demand - inventory_on_hand, min=0).sum() + self.holding_cost * torch.clip(inventory_on_hand - demand, min=0).sum()
        # update state
        state = self.update_state(state, action, demand)
        # return state and reward
        return state, reward
    
    def simulate_batch(self, model, demand_batch):
        """
        Simulate a batch of demand data
        """

        # initialize state as a matrix of zeros
        state = torch.zeros(demand_batch.shape[0], demand_batch.shape[1], self.lead_time).to(self.device)
        # print(f'initial state: {state.shape}')
        # initialize reward across batch
        batch_reward = 0
        reward_to_report = 0

        # loop through periods
        for period in range(self.periods):
            # get demand
            demand = demand_batch[:, :, period]
            # get action (i.e, order quantity)
            action = model({'x': state})
            # action = model(state)
            # print(f'state: {state[0]}')
            # print(f'action: {action[0]}')
            # print(f'demand: {demand[0]}')
            # print()
            # get new state and reward
            state, reward = self.step(state, action, demand)
            

            batch_reward += reward
            # add reward to batch reward only after lead time has passed (as costs on first periods do not depend on agent's actions)
            if period >= 20:
            # if period >= self.lead_time:
                reward_to_report += reward

        # return reward
        return batch_reward, reward_to_report

### Trainer class
Object that trains the model (i.e., Neural Net) making use of the data, optimizer (this object updates the parameters of the NN), and simulator

In [6]:
class Trainer():
    """
    Trainer class
    """

    def __init__(self, device='cpu'):
        
        self.all_train_losses = []
        self.all_test_losses = [] 
        self.device = device
    
    def reset(self):
        """
        Reset the losses
        """

        self.all_train_losses = []
        self.all_test_losses = []

    def train(self, epochs, model, train_loader, test_loader, optimizer, simulator):
        """
        Train the model
        """
        
        # number of periods for which we are tracking the loss
        periods_tracking_loss = 30
        # periods_tracking_loss = simulator.parameters['periods'] - simulator.parameters['lead_time']

        for epoch in range(epochs): # make multiple passes through the dataset

            epoch_loss = 0.0
            epoch_loss_to_report = 0.0
            test_loss_to_report = 0.0
            total_train_samples = 0
            total_test_samples = 0

            for i, demand_batch in enumerate(train_loader, 0):  # loop through batches of train data
                # move data to device
                demand_batch = demand_batch.to(self.device)

                # zero-out the gradient
                optimizer.zero_grad()

                # forward pass
                total_reward, reward_to_report = simulator.simulate_batch(model, demand_batch.float())
                epoch_loss += total_reward.item()
                epoch_loss_to_report += reward_to_report.item()
                total_train_samples += len(demand_batch)
                loss = total_reward/(len(demand_batch)*(demand_batch.shape[2]))
                # loss_to_report = total_reward/(len(demand_batch))

                # backward pass (to calculate gradient) and take gradient step
                loss.backward()
                optimizer.step()
            
            for i, demand_batch in enumerate(test_loader, 0):  # loop through batches of test data
                demand_batch = demand_batch.to(self.device)
                # forward pass
                total_reward, reward_to_report = simulator.simulate_batch(model, demand_batch.float())
                test_loss_to_report += reward_to_report.item()
                # test_loss += simulator.simulate_batch(model, demand_batch.float()).item()
                total_test_samples += len(demand_batch)

            average_train_loss = epoch_loss_to_report/(total_train_samples*periods_tracking_loss)
            average_test_loss = test_loss_to_report/(total_test_samples*periods_tracking_loss)

            self.all_train_losses.append(average_train_loss)
            self.all_test_losses.append(average_test_loss)

            # print epoch number and average per-period loss every 10 epochs
            if epoch % 10 == 9:
                print()
                print(f'epoch: {epoch + 1}')
                print(f'Average per-period train loss: {average_train_loss}')
                print(f'Average per-period test loss: {average_test_loss}')
            # print(f'test loss: {test_loss/(test_samples*(parameters["periods"] - parameters["lead_time"]))}')
            epoch_loss = 0.0
    
    def plot_losses(self, ymin=None, ymax=None):
        """
        Plot train and test losses for each epoch
        """

        plt.plot(self.all_train_losses, label='train loss')
        plt.plot(self.all_test_losses, label='test loss')
        plt.legend()

        if ymin is not None and ymax is not None:
            plt.ylim(ymin, ymax)
        plt.show()

## Parameter definition and object instantiation

In [7]:
demand_parameters = {'distribution': 'normal', 'mean': 5, 'std': 1.6} # mean and standard deviation of normal demand
train_seed = 42
test_seed = 43
periods = 50
train_samples = 2**13
test_samples = 2**13
data_params_train = {'num_samples': train_samples}
data_params_test = {'num_samples': test_samples}
problem_params = {'periods': periods, 'lost_demand': False}
store_params = {'lead_time': 4, 'underage_cost': 1, 'holding_cost': 0.5}
# demand_parameters, problem_params, data_params, seed=None
# create a DemandDataset object with specific mean, std, seeds, periods, and samples for train and test sets
train_demand_dataset = DemandDataset(demand_parameters, problem_params, data_params_train, train_seed)
test_demand_dataset = DemandDataset(demand_parameters, problem_params, data_params_test, test_seed)



In [8]:
print(train_demand_dataset[0].shape)

(1, 50)


In [13]:
train_batch_size = 2**10
test_batch_size = 2**10

# create DataLoader objects
train_loader = DataLoader(train_demand_dataset, batch_size=train_batch_size, shuffle=True)
test_loader = DataLoader(test_demand_dataset, batch_size=test_batch_size, shuffle=False)

In [10]:
simulator = Simulator(parameters, device=device).to(device)

NameError: name 'parameters' is not defined

In [11]:
parameters = {'periods': 50, 
              'initial_inventory': 0.0, 
              'underage_cost': 9.0, 
              'holding_cost': 1.0, 
              'lead_time': 4, 
              'stores': 1,
              'warehouses': 0,
              'lost_demand': False}

neurons_per_hidden_layer = [32, 32, 32]
lr = 0.003

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# create a simulator object with the following parameters
simulator = Simulator(parameters, device=device).to(device)
# create a fully connected neural network with LazyLinear layers (i.e., input size is inferred from data) and Tanh activation function
model = FullyConnectedNN(neurons_per_hidden_layer, output_size=parameters['stores']).to(device)
# create an Adam optimizer with specified learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# create a trainer
trainer = Trainer(device=device)

/user/ma4177/.conda/envs/MatiasRL/lib/python3.10/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


## Train the model and plot learning curves

In [14]:
# trainer.lr = 0.1
# You can run this cell multiple times, and the training will be resumed from the previous state (unless you re-run the cell that instantiates the Neural Net)
epochs = 1000
# train the model for many epochs
trainer.train(epochs, model, train_loader, test_loader, optimizer, simulator)


epoch: 10
Average per-period train loss: 6.329889043172201
Average per-period test loss: 6.2838900248209635

epoch: 20
Average per-period train loss: 6.30523370107015
Average per-period test loss: 6.259692192077637


KeyboardInterrupt: 

In [ ]:
# plot the losses
ymin, ymax = 0, 100  # set the y-axis limit for the plot
trainer.plot_losses(ymin=ymin, ymax=ymax)